In [1]:
from ckks_package.ckks import *

In [2]:
N = 2**15  # Ring degree
n = 2**3  # Number of slots
L_boot = 19  # Maximal level during bootstrapping
q0 = 2**29  # Smallest modulus
p = 2**23  # Scaling factor outside of bootstrapping
delta = q0  # Scaling factor during bootstrapping

# The grouping parameter for bootstrapping is log_radix. It lies between 1 and
# log(n, 2). The smaller the log_radix, the faster the bootstrapping, but the
# more levels are consumed.
log_radix = log(n, 2)

CKKS.config(N, n, L_boot, q0, p, delta)
CKKS.key_gen()
CKKS.config_bootstrap(CKKS.sk, log_radix)

The CKKS configuration is done!

The key generation is done!
Estimated security: 2^(91.9) operations.

Encoding relevant vectors as polynomials...
Generating matrices required for CoeffToSlot and SlotToCoeff...
Encoding these matrices as polynomials...
Generating missing switching keys...
The bootstrapping configuration is done!



In [3]:
# Creating two random complex vectors

complex_vectors = [
    np.array([randint(-3, 3) + 1j * randint(-3, 3) for _ in range(n)])
    for _ in range(2)
]
complex_vectors

[array([ 3.-1.j,  0.+0.j, -1.+1.j,  1.+2.j, -1.-2.j, -0.-1.j,  1.+3.j,
         1.+1.j]),
 array([-0.-2.j,  2.-2.j,  2.-2.j,  2.+3.j, -3.-2.j,  1.+0.j, -1.-2.j,
         2.-2.j])]

In [4]:
# Encoding as polynomials

plaintext_polys = [CKKS.encode(z) for z in complex_vectors]
plaintext_polys

[4194304 + 6255302X^2048 - 3943879X^4096 + 2667530X^6144 - 741455X^8192 + 1390492X^10240 - 6311185X^12288 + 4948834X^14336 + 3145728X^16384 - 801413X^18432 - 4041244X^20480 + 217467X^22528 - 741455X^24576 - 6637573X^26624 - 9424001X^28672 - 1153853X^30720 mod(2^581),
 5242880 - 374964X^2048 - 6380033X^4096 - 5711797X^6144 - 11863283X^8192 + 5601838X^10240 - 1370031X^12288 + 1594457X^14336 - 9437184X^16384 + 4793904X^18432 + 3777667X^20480 - 3533796X^22528 + 1482910X^24576 - 5390750X^26624 + 567485X^28672 - 8938049X^30720 mod(2^581)]

In [5]:
# Encrypting

ciphertexts = [CKKS.enc_poly_with_sk(pt, CKKS.sk) for pt in plaintext_polys]
ciphertexts

[A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).,
 A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).]

In [6]:
# Checking correctness

for i in range(2):
    pt = ciphertexts[i].dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 3.00000050e+00-9.99999108e-01j  6.62178063e-07-1.12392460e-06j
 -1.00000083e+00+9.99999941e-01j  9.99999627e-01+1.99999969e+00j
 -9.99999377e-01-2.00000143e+00j  1.26465060e-06-1.00000086e+00j
  1.00000018e+00+2.99999930e+00j  9.99998923e-01+1.00000073e+00j]
[ 7.59433663e-07-2.00000085e+00j  1.99999926e+00-2.00000048e+00j
  1.99999969e+00-2.00000071e+00j  2.00000131e+00+3.00000049e+00j
 -2.99999747e+00-2.00000060e+00j  1.00000018e+00-1.21951174e-07j
 -1.00000087e+00-2.00000002e+00j  2.00000095e+00-2.00000058e+00j]


In [7]:
# Homomorphic operations

ct_add = ciphertexts[0] + ciphertexts[1]
ct_mul = (
    ciphertexts[0] @ ciphertexts[1]
)  # Polynomial multiplication followed by a rescaling operation
print(ct_add)
print(ct_mul)

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^24 (level 24 out of 24).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^23 (level 23 out of 24).


In [8]:
# Checking correctness

for ct in [ct_add, ct_mul]:
    pt = ct.dec_to_poly(CKKS.sk)
    print(CKKS.decode(pt))

[ 3.00000126e+00-2.99999995j  1.99999992e+00-2.0000016j
  9.99998866e-01-1.00000077j  3.00000094e+00+5.00000018j
 -3.99999685e+00-4.00000203j  1.00000144e+00-1.00000098j
 -6.91075320e-07+0.99999928j  2.99999987e+00-0.99999985j]
[-1.99999670e+00-6.00000494e+00j -4.82282000e-07-3.16196963e-06j
 -6.12504557e-07+4.00000166e+00j -4.00000030e+00+7.00000112e+00j
 -1.00000838e+00+7.99999791e+00j  7.13663227e-07-1.00000194e+00j
  4.99999770e+00-5.00000262e+00j  4.00000138e+00+3.39893854e-06j]


In [9]:
# Bootstrapping

ct = ciphertexts[0] % q0  # Project to the lowest level l = 0
print(ct)

ct_boot = ct.bootstrap(log_radix)
print(ct_boot)

pt_boot = ct_boot.dec_to_poly(CKKS.sk)
print(CKKS.decode(pt_boot))

print(ct.get_poly_precision(ct_boot, CKKS.sk))

A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^0 (level 0 out of 24).
A CKKS ciphertext with degree N = 2^15 and modulus q = (2^29) * (2^23)^1 (level 1 out of 24).
[ 2.99693335e+00-9.98337705e-01j  1.16193570e-03+1.58095425e-03j
 -9.97975344e-01+9.99632702e-01j  9.98377218e-01+1.99722171e+00j
 -1.00080515e+00-1.99835994e+00j  7.64526031e-04-9.97571346e-01j
  1.00105609e+00+2.99726604e+00j  9.98265311e-01+9.98500832e-01j]
11.348326225443161
